In [46]:
pip show numpy


Name: numpy
Version: 1.22.0
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: c:\users\dell\anaconda3\envs\fer_env\lib\site-packages
Requires: 
Required-by: albumentations, contourpy, facenet-pytorch, fer-pytorch, h5py, imageio, matplotlib, mkl_fft, mkl_random, ml_dtypes, moviepy, opencv-python, opencv-python-headless, pandas, pytorch-lightning, PyWavelets, scikit-image, scikit-learn, scipy, tensorboard, tifffile, torchmetrics, torchvision
Note: you may need to restart the kernel to use updated packages.


c:\Users\dell\anaconda3\envs\fer_env\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
c:\Users\dell\anaconda3\envs\fer_env\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
c:\Users\dell\anaconda3\envs\fer_env\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [71]:
import cv2
import torch
from fer_pytorch.fer import FER

# Check OpenCV version
print(f"OpenCV version: {cv2.__version__}")

# Check Torch version
print(f"Torch version: {torch.__version__}")

# Initialize FER model and test
fer_model = FER()
print("FER model initialized successfully")

# Test FER on a sample image
img_path = 'path_to_your_image.jpg'  # Provide the path to a sample image file
img = cv2.imread(img_path)

if img is not None:
    result = fer_model.predict_image(img, show_top=True)
    print("Emotion prediction result:", result)
else:
    print("Failed to load image. Make sure the path is correct.")

# Simple PyTorch tensor test
x = torch.rand(5, 3)
print("Sample PyTorch tensor:\n", x)


OpenCV version: 4.5.5
Torch version: 1.11.0+cpu
FER model initialized successfully
Failed to load image. Make sure the path is correct.
Sample PyTorch tensor:
 tensor([[0.4220, 0.2840, 0.5680],
        [0.3895, 0.4011, 0.7885],
        [0.3728, 0.6350, 0.0621],
        [0.9981, 0.0773, 0.2424],
        [0.1329, 0.3789, 0.2229]])


In [48]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)


1.4.2
1.22.0


In [49]:
import fer_pytorch
print(fer_pytorch.__version__)


1.1.2


In [50]:
import json
import os
import warnings
from typing import Any, Dict, List, Optional, Union

import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from facenet_pytorch import MTCNN
from PIL import Image
from tqdm import tqdm

from fer_pytorch.model import FERModel
from fer_pytorch.pre_trained_models import get_pretrained_model
warnings.simplefilter(action="always")


In [63]:

# Initialize the MTCNN for face detection and the FER model for emotion recognition
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mtcnn = MTCNN(keep_all=True, device=device)
fer_model = FER()



In [66]:
# Emotion dictionary to map indices to emotion labels
EMOTION_DICT = {0: "neutral", 1: "happiness", 2: "surprise", 3: "sadness", 4: "anger", 5: "disgust", 6: "fear"}

# Function to process a video and perform emotion recognition
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Cannot open video file: {video_path}")

    frame_count = 0
    results = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect faces using MTCNN
        boxes, _ = mtcnn.detect(frame)

        # Process each detected face
        if boxes is not None:
            for (x, y, w, h) in boxes:
                face = frame[int(y):int(h), int(x):int(w)]
                face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

                # Convert face to tensor and predict emotion
                face_tensor = torch.from_numpy(np.transpose(face_rgb, (2, 0, 1))).unsqueeze(0).float().to(device)
                with torch.no_grad():
                    emotion_output = fer_model.predict_image(face_rgb)
                
                # Store the result
                if emotion_output:
                    top_emotion = emotion_output[0]['top_emotion']
                    emotion_label = list(top_emotion.keys())[0]
                    emotion_prob = list(top_emotion.values())[0]
                    results.append({
                        'frame': frame_count,
                        'emotion': emotion_label,
                        'box': [x, y, w, h],
                        'probability': emotion_prob
                    })

        frame_count += 1

    cap.release()
    return results

# Process your video file and print results
video_path = "C:\\Users\\dell\\Desktop\\graduation project\\AI04-Grad\\datasets\\CREMA-D\\crema-d videos\\1002_DFA_ANG_XX.mp4"
results = process_video(video_path)
for result in results:
    print(f"Frame {result['frame']}: {result['emotion']} ({result['probability']:.2f}) at {result['box']}")


c:\Users\dell\anaconda3\envs\fer_env\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
c:\Users\dell\anaconda3\envs\fer_env\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
c:\Users\dell\anaconda3\envs\fer_env\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
c:\Users\dell\anaconda3\envs\fe

In [79]:
import cv2
from facenet_pytorch import MTCNN
import torch

# Path to your video file
video_filename = r"C:\Users\dell\Desktop\graduation project\AI04-Grad\datasets\CREMA-D\1002_DFA_HAP_XX.mp4"

# Initialize the video capture object
cap = cv2.VideoCapture(video_filename)

# Initialize the MTCNN face detector
mtcnn = MTCNN(keep_all=True, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# Loop through video frames
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Detect faces in the current frame
    boxes, _ = mtcnn.detect(frame)
    
    # Draw bounding boxes on the frame
    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
    # Display the video with face detections in real-time
    cv2.imshow('Face Detection', frame)
    
    # Break if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()


In [78]:
import cv2
from fer_pytorch.fer import FER

# Path to your video file
video_filename = r"C:\Users\dell\Desktop\graduation project\AI04-Grad\datasets\CREMA-D\1002_DFA_HAP_XX.mp4"

# Initialize the video capture object
cap = cv2.VideoCapture(video_filename)

# Initialize the FER detector
fer_detector = FER()

# Read a single frame
ret, frame = cap.read()

if ret:
    # Detect emotions on the entire frame (or you can crop to a face region)
    emotions = fer_detector.detect_emotions(frame)
    
    if emotions:
        print("Detected emotions:", emotions)
    else:
        print("No emotions detected.")

# Release the video capture object
cap.release()


In [84]:
import cv2

# Path to your video file
video_filename = 'C:\\Users\\dell\\Desktop\\graduation project\\AI04-Grad\\datasets\\CREMA-D\\crema-d videos\\1002_DFA_ANG_XX.mp4'

# Initialize the video capture object
cap = cv2.VideoCapture(video_filename)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video opened successfully.")
    
# Loop through video frames
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("End of video or failed to grab frame.")
        break
    
    # Display the frame to confirm video is being read
    cv2.imshow('Video Frame', frame)
    
    # Break if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()


Video opened successfully.
End of video or failed to grab frame.


In [85]:
# Get total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total number of frames: {total_frames}")


Total number of frames: 0


In [82]:
import cv2

# Path to the uploaded video file
video_filename = 'C:\\Users\\dell\\Desktop\\graduation project\\AI04-Grad\\datasets\\CREMA-D\\crema-d videos\\1002_DFA_ANG_XX.mp4'

# Initialize the video capture object
cap = cv2.VideoCapture(video_filename)

# Check if the video opened successfully
video_opened = cap.isOpened()

# Clean up by releasing the video object
cap.release()

video_opened


True

In [86]:

# Path to the uploaded video file
video_filename = 'C:\\Users\\dell\\Desktop\\graduation project\\AI04-Grad\\datasets\\CREMA-D\\crema-d videos\\1002_DFA_ANG_XX.mp4'

# Initialize the video capture object
cap = cv2.VideoCapture(video_filename)

# Get total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Clean up by releasing the video object
cap.release()

total_frames


78